### Datensatz für YOLOv5-Format splitten
Hierfür wird der selbst generierte Datensatz für das YOLOv5 Training in YOLO Training in 80% Training und 20% Validierungsdatensatz getrennt. Nach der Aufteilung ist es wichtig, sicherzustellen, dass für beide Klassen, nämlich "Open Hand" und "Fist", ausreichend Bilder vorhanden sind. Ein ausgewogenes Verhältnis zwischen den Klassen ist entscheidend, um ein effektives Training zu gewährleisten.

**YOLO-Format**
- Die erste Spalte enthält die Klassenindizes.
- Die zweite bis fünfte Spalte enthält die normalisierten Koordinaten der Bounding Box (x, y, w, h).

In [ ]:
import os
from sklearn.model_selection import train_test_split
import shutil

In [7]:
# Pfade zu den Ordnern mit Bildern und Labels
img_folder = 'dataset/img_merged'
labels_folder = 'dataset/labels'

# Liste der Dateinamen in den Ordnern
img_filenames = os.listdir(img_folder)
label_filenames = os.listdir(labels_folder)

# Entferne Dateiendungen (z.B. '.jpg', '.txt')
img_names = [name.split('.')[0] for name in img_filenames]
label_names = [name.split('.')[0] for name in label_filenames]

# Überprüfe Übereinstimmung der Dateinamen
assert set(img_names) == set(label_names), "Bild- und Labelnamen stimmen nicht überein!"

# Aufteilung in Trainings- und Validierungsdaten
train_img, val_img, train_labels, val_labels = train_test_split(img_names, label_names, test_size=0.2, random_state=42)

# Verzeichnis für den gesamten Datensatz erstellen
output_dir = '../yolov5_training/dataset'
os.makedirs(output_dir, exist_ok=True)

# Verzeichnisse für Bilder und Beschriftungen erstellen
train_img_dir = os.path.join(output_dir, 'img/train')
train_labels_dir = os.path.join(output_dir, 'labels/train')
val_img_dir = os.path.join(output_dir, 'img/val')
val_labels_dir = os.path.join(output_dir, 'labels/val')

os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Kopiere Dateien in die entsprechenden Verzeichnisse
for img_name, label_name in zip(train_img, train_labels):
    shutil.copy(os.path.join(img_folder, img_name + '.jpg'), train_img_dir)
    shutil.copy(os.path.join(labels_folder, label_name + '.txt'), train_labels_dir)

for img_name, label_name in zip(val_img, val_labels):
    shutil.copy(os.path.join(img_folder, img_name + '.jpg'), val_img_dir)
    shutil.copy(os.path.join(labels_folder, label_name + '.txt'), val_labels_dir)

print("Aufteilung in Trainings- und Validierungsdaten abgeschlossen.")


Aufteilung in Trainings- und Validierungsdaten abgeschlossen.


**Erstellung einer classes.txt**
- Enthält Klassennamen und Klassenindiz

In [4]:
# Pfade zu den Ordnern mit Bildern und Labels
img_folder = 'dataset/img_merged'
labels_folder = 'dataset/labels'

# Liste der Dateinamen in den Ordnern
img_filenames = os.listdir(img_folder)
label_filenames = os.listdir(labels_folder)

# Klassen und ihre Indizierung definieren
classes = {
    'Open_Hand': 0,
    'Fist': 1
}

# Inhalt für die classes.txt-Datei erstellen
classes_content = "\n".join([f"{class_idx} {class_name}" for class_name, class_idx in classes.items()])

# Verzeichnis für die classes.txt-Datei in Trainings- und Validierungsdaten erstellen
output_dir_train = '../yolov5_training/dataset/labels/train'
output_dir_val = '../yolov5_training/dataset/labels/val'

os.makedirs(output_dir_train, exist_ok=True)
os.makedirs(output_dir_val, exist_ok=True)

# classes.txt-Datei in Trainings- und Validierungsordnern erstellen
classes_file_path_train = os.path.join(output_dir_train, 'classes.txt')
classes_file_path_val = os.path.join(output_dir_val, 'classes.txt')

with open(classes_file_path_train, 'w') as classes_file_train:
    classes_file_train.write(classes_content)

with open(classes_file_path_val, 'w') as classes_file_val:
    classes_file_val.write(classes_content)

print("classes.txt-Datei erstellt in Trainings- und Validierungsordnern.")


classes.txt-Datei erstellt in Trainings- und Validierungsordnern.


**YAML-Datei**
Die YAML-Datei ist eine Konfigurationsdatei für das Training des YOLO-Modells. Diese ist wie folgt aufgebaut:
- `train`: Pfad zum Verzeichnis, das die Trainingsbilder enthält.
- `val`: Pfad zum Verzeichnis mit den Validierungsbildern.
- `nc`: Anzahl der Klassen im Datensatz.
- `names`: Eine Liste der Klassennamen. 
- `patience`: Anzahl der Epochen, die gewartet werden, wenn die Leistung des Modells nicht besser wird, bevor das Training beendet wird. Hierbei kann bei Overfitting das Training automatisch abgebrochen werden.
- `delta`: Schwellenwert, um zu bestimmen, ob die Leistung des Modells sich verbessert hat. Wenn die Verbesserung kleiner als dieser Schwellenwert ist, wird das Training gestoppt. 

In [4]:
data_path = "../yolov5_training/dataset/data.yaml"
train_path = "../../yolov5_training/dataset/images/train/"
val_path = "../../yolov5_training/dataset/images/val/"
class_names_file = "../yolov5_training/dataset/labels/train/classes.txt"

with open(class_names_file, 'r') as f:
    class_names = [line.strip().split(' ', 1)[1] for line in f.readlines()]

data_yaml = f"train: {train_path}\n" \
            f"val: {val_path}\n" \
            f"nc: {len(class_names)}\n" \
            f"names: {class_names}\n" \
            f"patience: 5  # Anzahl der Epochen, die gewartet werden, wenn die Leistung nicht besser wird\n" \
            f"delta: 0.0001  # Schwellenwert, um zu bestimmen, ob die Leistung sich verbessert hat\n" 

with open(data_path, 'w') as file:
    file.write(data_yaml)

print(f"Die data.yaml-Datei wurde erfolgreich erstellt: {data_path}")

Die data.yaml-Datei wurde erfolgreich erstellt: ../yolov5_training/dataset/data.yaml
